In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
import time
import random
import glob
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.utils.data import Dataset, DataLoader, DistributedSampler
from torch.multiprocessing.spawn import spawn
import torchvision.models as models
import torchvision.datasets as dst
from torchvision.io import read_image
from torchvision.transforms import v2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# data
def transform_image(img):
    transforms = v2.Compose([
        # v2.Resize(224),
        # v2.CenterCrop(224),
        v2.ToTensor(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(
            mean=[0.485, 0.456, 0.406],
            std =[0.229, 0.224, 0.225]
        )
    ])
    return transforms(img)

test_dataset = dst.CIFAR10(
    root="../data",
    train=False,
    download=False,
    transform=transform_image
)

dataloader = DataLoader(test_dataset, batch_size=200, shuffle=False)

# model
vgg = models.vgg16(pretrained=True).to(device).eval()
vgg.classifier = nn.Sequential(*list(vgg.classifier.children())[:4])

In [10]:
with torch.no_grad():
    imgs, labels = next(iter(dataloader))
    imgs = imgs.to(device)               # Tensor[100,4096]
    feats = vgg(imgs)
feats_np = feats.cpu().numpy()   

In [11]:
# 95% 累積寄与率
pca95 = PCA(n_components=0.95)
X95 = pca95.fit_transform(feats_np)
dim95 = X95.shape[1]

# 90% 累積寄与率
pca90 = PCA(n_components=0.90)
X90 = pca90.fit_transform(feats_np)
dim90 = X90.shape[1]

# 固定 128 次元
pca128 = PCA(n_components=128)
X128 = pca128.fit_transform(feats_np)

In [12]:
settings = {
    f"原始(4096)": feats_np,
    f"95%寄与率({dim95}次元)": X95,
    f"90%寄与率({dim90}次元)": X90,
    "固定128次元": X128,
}

In [15]:
settings[f"95%寄与率({dim95}次元)"]

array([[-1.6178340e+02,  8.6934746e+01, -7.5167198e+01, ...,
        -3.2718880e+01, -2.4478140e+01,  1.7496986e+01],
       [-3.1336496e+01,  1.4742610e+02, -3.7529450e-02, ...,
        -2.6084763e+01, -1.0323290e+01, -1.9575739e+01],
       [ 1.7347859e+02,  2.6521482e+01,  1.6934414e+01, ...,
         6.8710756e+00,  1.2091462e+01,  1.9301725e+01],
       ...,
       [ 2.0207272e+02, -2.0509333e+01,  2.5992802e+01, ...,
         1.4902886e+01,  1.0071152e+01,  1.4660802e+01],
       [-3.2453677e+02, -1.3707321e+02, -2.3341423e+02, ...,
        -1.0337401e+01,  9.8100557e+00, -8.8760698e-01],
       [ 6.2061562e+01,  4.1045174e+01,  3.2111359e+00, ...,
        -3.4509530e+00,  1.3311825e+01, -1.4658801e+01]], dtype=float32)

In [17]:
dim90

32

In [19]:
from sklearn.metrics import silhouette_score
classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

for name, X in settings.items():
    print(f"\n>>> {name} の場合:")
    for k in [5, 10]:
        km = KMeans(n_clusters=k, random_state=0).fit(X)
        cluster_labels = km.labels_
        # 真のラベルごとにクラスタ分布を集計
        df = pd.crosstab(pd.Series(labels, name='TrueLabel'),
                        pd.Series(cluster_labels, name='Cluster'))
        # インデックスをラベル名に変換
        df.index = [classes[i] for i in df.index]

        print(f"--- k = {k} ---")
        print(df)


>>> 原始(4096) の場合:
--- k = 5 ---
Cluster     0   1   2   3   4
airplane    5   4   0   0   0
automobile  0   0   0   8   0
bird        0   2   3   4   6
cat         0   0  46   1   0
deer        0   0  23   0  21
dog         0   0   3  20   0
frog        1   0   0   0   0
horse       0   3   0  17   1
ship        3   2   0   2   0
truck       0  16   0   0   9
--- k = 10 ---
Cluster     0  1   2   3  4   5  6  7   8   9
airplane    2  0   0   0  0   0  0  7   0   0
automobile  0  8   0   0  0   0  0  0   0   0
bird        0  3  10   0  0   1  0  0   1   0
cat         0  0   1  46  0   0  0  0   0   0
deer        0  0   1   1  0  40  0  0   1   1
dog         0  2  20   0  0   0  0  0   1   0
frog        0  0   0   0  1   0  0  0   0   0
horse       0  2   0   0  0   0  2  0  17   0
ship        1  0   0   0  0   0  3  1   2   0
truck       3  0   0   0  0   0  0  1   1  20

>>> 95%寄与率(49次元) の場合:
--- k = 5 ---
Cluster     0   1   2   3   4
airplane    5   4   0   0   0
automobile  0   0  

In [20]:
evr128 = pca128.explained_variance_ratio_
evr128.sum()

np.float32(0.9957808)